In [1]:
!pwd

/home/sagemaker-user/Brain-Tumor-Classification-MRI/notebooks


In [2]:
%cd ..

/home/sagemaker-user/Brain-Tumor-Classification-MRI


# Create Model Configuration

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass
class ModelConfig:
    config_root_dir: Path
    config_model_path: Path
    config_updated_model_path: Path
    params_image_size: list
    params_lr: float
    params_include_top: bool
    params_weights: str
    params_classes: int

# Create Configuration Manager

In [5]:
from src.utils import read_yaml, create_dirs
from src.constants import *
import os

In [6]:
class ConfigManager:
    def __init__(self, config_path=CONFIG_PATH, params_path=PARAMS_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.artifacts_folder = self.config.artifacts_folder
        create_dirs([self.artifacts_folder])

    def get_model_config(self):
        model_config = self.config.model
        model_params = self.params
        create_dirs([model_config.model_folder_name])
        return ModelConfig(
            config_root_dir = Path(os.path.join(self.artifacts_folder, model_config.model_folder_name)),
            config_model_path = Path(os.path.join(self.artifacts_folder, model_config.model_path)),
            config_updated_model_path = Path(os.path.join(self.artifacts_folder, model_config.updated_model_path)),
            params_image_size = model_params.IMAGE_SIZE,
            params_lr = model_params.LEARNING_RATE,
            params_include_top = model_params.INCLUDE_TOP,
            params_weights = model_params.WEIGHTS,
            params_classes = model_params.CLASSES
        )

# Download The Model

In [7]:
import tensorflow as tf

2024-09-28 07:03:25.329329: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-28 07:03:25.356904: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
class Model:
    def __init__(self, config: ModelConfig):
        self.config = config

    
    def download(self):
        self.model = tf.keras.applications.vgg19.VGG19(
            input_shape = self.config.params_image_size,
            include_top = self.config.params_include_top,
            weights = self.config.params_weights
        )
        self.model.save(self.config.config_model_path)

    
    @staticmethod
    def _prepare(model, freeze_all=True, freeze_till=None, learning_rate=0.01, classes=4):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten = tf.keras.layers.Flatten()(model.output)
        output_dense = tf.keras.layers.Dense(classes, activation='softmax')(flatten)
        final_model = tf.keras.models.Model(
            inputs=model.input, outputs=output_dense
        )

        final_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"]
        )
        final_model.summary()

        return final_model


    def update(self):
        self.final_model = self._prepare(
            model = self.model,
            freeze_all = False,
            freeze_till = None,
            learning_rate = self.config.params_lr,
            classes = self.config.params_classes
        )
        self.final_model.save(self.config.config_updated_model_path)

# Perform The Step

In [9]:
try:
    config = ConfigManager()
    model_config = config.get_model_config()
    model = Model(model_config)
    model.download()
    model.update()
except Exception as e:
    raise e

[2024-09-28 07:03:27,109: INFO: utils: Directory 'model' was successfully created...]
[2024-09-28 07:03:27,514: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     